# Exploración Inmuebles
Notebook que incluye la exploración del dataset "indices_institucional_2005_2022" para la posterior unión con el dataset "base_indices_2005-2023".

Requiere los resultados del notebook draft-df.

In [ ]:
from pathlib import Path
import joblib
from typing import List

import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline

from pipeline import (
    DropHighNAPercentage,
    DropColumns,
    NanInputer,
)
from utils import (
    show,
)

pd.set_option("display.max_columns", None)

In [ ]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data"

DATA_PATH = DATA_DIR / 'indices_institucional_2005_2022.xlsx'
DATA_PATH.exists()

### Loading Data

In [ ]:
# xlsx to pkl
# inmuebles = pd.read_excel(DATA_PATH, sheet_name="Inmuebles")
# labs = pd.read_excel(DATA_PATH, sheet_name="Laboratorios y Talleres")
# docentes = pd.read_excel(DATA_PATH, sheet_name="Docentes")

# extra_data = inmuebles, labs, docentes
# joblib.dump(extra_data, DATA_DIR / 'extra_data.pkl')

In [ ]:
inmuebles, labs, docentes = joblib.load(DATA_DIR / 'extra_data.pkl')
df = joblib.load(DATA_DIR / 'processed_df.pkl')

### Preprocessing Data

In [ ]:
def prepare_for_merge(df_to_merge: pd.DataFrame, year_col: str) -> pd.DataFrame:
    df_to_merge["Año"] = df_to_merge[year_col].astype(int)
    df_to_merge = df_to_merge.drop(columns=[year_col])
    df_to_merge = df_to_merge.rename(columns={"idSede": "Cód. Sede"})
    df_to_merge["Cód. Sede"] = df_to_merge["Cód. Sede"].astype(int)
    df_to_merge["Cód. Institución"] = df_to_merge["Cód. Institución"].astype(int)
    df_to_merge = df_to_merge.drop(columns=["Tipo Institución", "Cód. Institución"])

    return df_to_merge


inmuebles = prepare_for_merge(inmuebles, year_col="Año Información")
labs = prepare_for_merge(labs, year_col="Año Proceso")
docentes = prepare_for_merge(docentes, year_col="Año Proceso")

In [ ]:
# pipes
inmuebles_pipe = Pipeline(
    [
        (
            "drop_columns",
            DropColumns(
                columns=["Nombre Institución", "Sede", "idInstitucion"], errors="ignore"
            ),
        ),
        ("drop_high_na", DropHighNAPercentage(na_threshold=0.3, exclude=[])),
        ("inputer", NanInputer(n_neighbors=5)),
    ]
)

labs_pipe = Pipeline(
    [
        (
            "drop_columns",
            DropColumns(
                columns=["Nombre Institución", "Sede", "idInstitucion"], errors="ignore"
            ),
        ),
        ("drop_high_na", DropHighNAPercentage(na_threshold=0.3, exclude=[])),
        ("inputer", NanInputer(n_neighbors=5)),
    ]
)

docentes_pipe = Pipeline(
    [
        (
            "drop_columns",
            DropColumns(
                columns=["Nombre Institución", "Sede", "idInstitucion"], errors="ignore"
            ),
        ),
        ("drop_high_na", DropHighNAPercentage(na_threshold=0.3, exclude=[])),
        ("inputer", NanInputer(n_neighbors=5)),
    ]
)

In [ ]:
inmuebles = inmuebles_pipe.fit_transform(inmuebles)
labs = labs_pipe.fit_transform(labs)
docentes = docentes_pipe.fit_transform(docentes)

In [ ]:
show(inmuebles)
show(labs)
show(docentes)
show(df)

In [ ]:
print(f"Total number of columns: {inmuebles.shape[1] + labs.shape[1] + docentes.shape[1]}")

In [ ]:
MERGE_COLS = [
    "Año",
    "Cód. Sede",
]

In [ ]:
df_to_merge = docentes

X_values = df[MERGE_COLS].to_records(index=False).tolist()
Y_values = df_to_merge[MERGE_COLS].to_records(index=False).tolist()

X = set(X_values)
Y = set(Y_values)

In [ ]:
len(X), len(Y), len(X & Y), len(X - Y), len(Y - X)

In [ ]:
X_tuples = df[MERGE_COLS].apply(lambda x: tuple(x), axis=1)
notna_perc = X_tuples[X_tuples.isin(Y)].shape[0] / X_tuples.shape[0]
print(f"Percentage of not null values: {notna_perc * 100:.2f}%")

## Merging Data

In [ ]:
def merge_dataframes(
    target_df: pd.DataFrame,
    df_to_merge: pd.DataFrame,
    merge_cols: List[str],
) -> pd.DataFrame:
    """
    Merge two dataframes on the specified columns,
    conserving the index of the target dataframe
    Args:
        target_df (pd.DataFrame): dataframe to merge with
        df_to_merge (pd.DataFrame): dataframe to merge
        merge_cols (List[str]): columns to merge on
    Returns:
        pd.DataFrame: merged dataframe
    """
    target_df = target_df.copy()
    
    target_df["index"] = target_df.index
    df_merged = target_df.merge(df_to_merge, on=merge_cols, how="inner")
    df_merged = df_merged.set_index("index")
    df_merged.index.name = None
    return df_merged

In [ ]:
df_merged = merge_dataframes(df, inmuebles, MERGE_COLS)
df_merged = merge_dataframes(df_merged, labs, MERGE_COLS)
df_merged = merge_dataframes(df_merged, docentes, MERGE_COLS)

In [ ]:
show(df_merged)